## 4D CT Sensitivity

This cell calculates the **sensitivity** of 4D CT scans:

**Sensitivity = True Positives / (True Positives + False Negatives)**

- **True Positives**: Scans with Exact or Side match
- **False Negatives**: Scans labeled as *Non-localizing*

In [ ]:
import pandas as pd 
preop_imaging_df = pd.read_excel('../../Data/final_imaging_data.xlsx')

In [2]:
# Define modality
modality = '4D CT Scan'

# Apply extract_side function if not already done to get imaging sides
# preop_imaging_df['4D CT Imaging Side'] = preop_imaging_df[modality].apply(extract_side)

# Filter to scans that localized anywhere (Left, Right)
localized_4dct = preop_imaging_df[preop_imaging_df['4D CT Imaging Side'].isin(['Left', 'Right'])].copy()
nonlocalized_4dct = preop_imaging_df[preop_imaging_df[modality] == 'Non-localizing'].copy()
# total localized and non-localized sestamibi
# localized_4dct['Calculated Concordance'] =localized_4dct.apply(check_concordance, axis=1)


true_positives = localized_4dct['4D CT Scan Calculated Concordance'].isin(['Yes (Exact)', 'Yes (Side)']).sum()
false_negatives = nonlocalized_4dct.shape[0]
false_negatives 
true_positives



125

In [3]:
sensitivity = true_positives / (true_positives + false_negatives)
print(f"4D CT Sensitivity: {sensitivity:.2%} ({true_positives} True Positive / {true_positives + false_negatives} total with disease)")

4D CT Sensitivity: 69.44% (125 True Positive / 180 total with disease)
